In [ ]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout

# 랜덤시드 고정시키기
np.random.seed(3)

In [ ]:
# 1. 데이터셋 생성하기
train_datagen = ImageDataGenerator(rescale=1./255,
                                   
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.7,
                                   zoom_range=[0.9, 2.2],
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')


train_generator = train_datagen.flow_from_directory('data/train',
                                                    target_size=(64,64),
                                                    batch_size=3,
                                                    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('data/test',
                                                  target_size=(64,64),
                                                  batch_size=3,
                                                  class_mode='categorical')
print(train_generator[0][:2])

In [ ]:
# 2. 모델 구성하기
model = Sequential()
model.add(Conv2D(4, kernel_size=(3,3),
                activation='relu',
                input_shape=(64,64,3)))


model.add(Conv2D(16, (3,3), activation='relu',padding='same'))
model.add(Dropout(0,2))

model.add(Conv2D(16, (3,3), activation='relu',padding='same'))
model.add(Dropout(0,2))


model.add(Dropout(0.2))

model.add(Conv2D(32, (3,3), activation='relu',padding='same'))
model.add(Dropout(0,2))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(6, activation='softmax'))

In [ ]:
# 3. 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
# 4. 모델 학습시키기
history = model.fit_generator(train_generator,
                              steps_per_epoch=128,
                              epochs=50,
                              validation_data=test_generator,
                              validation_steps=6)

In [ ]:
# 5. 모델 평가하기
print("--Evaluate--")
scores = model.evaluate_generator(test_generator, steps=6)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
# 6. 모델 사용하기
print("-- Predict --")

output = model.predict_generator(
            test_generator, 
            steps = 5)

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

print(output)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:




acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('data_classifier3.h5')